In [1]:
import pandas as pd
import ast
from textblob import TextBlob
import re
import math
import matplotlib.pyplot as plt
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import datetime
from datetime import timedelta
from matplotlib.pyplot import MultipleLocator

In [2]:
#load dataframe
tws_df = pd.read_csv('/home/zhoulita/scratch/covid_data_unprocessed/dataFrame/tweet_df_loc_total.csv',low_memory=False,lineterminator='\n')

def get_week_index(df):
    return int((df['day_index']-1)/7)
tws_df['week_index']= tws_df.apply(get_week_index,axis = 1)
#remove unnecessary column
tws_df.drop(['lang', 'referenced_tweets','author_id', 'source', 'possibly_sensitive', 'public_metrics','entities', 'context_annotations', 'in_reply_to_user_id', 'attachments'], axis=1, inplace=True)
tws_df.drop(['Unnamed: 0', 'Unnamed: 0.1','geo_x', 'id_y', 'withheld'], axis=1, inplace=True)

In [5]:
#load the us places with the state code
us_pls_df = pd.read_csv("/home/zhoulita/scratch/covid_19_tweets/us_places.csv")
us_pls_df.drop(['Unnamed: 0', 'full_name','place_type', 'country_code', 'country','geo','name','Lontitude','Latitude'], axis=1, inplace=True)

In [4]:
total_df = pd.merge(tws_df,us_pls_df,how = 'inner',left_on = 'geo_id',right_on = 'id')
total_df.drop(['place_type', 'country','place_type', 'Lontitude', 'geo_y','Latitude','id'], axis=1, inplace=True)

In [7]:
#text cleaning
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)



#define a function to clean the text
def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text)#remove the @
    text = re.sub(r'#','',text) #remove the #
    text = re.sub(r'RT[\s]+','',text) #remove retweet
    text = re.sub(r'https?:\/\/\S+','',text) #remove http 
    text =  re.sub('\n','',text)
    text = re.sub('\S*@\S*\s?', '', text)#remove email
    text = re.sub('\s+', ' ', text)
    text = remove_emoji(text)
    return text

total_df['processed_text'] = total_df['text'].apply(clean_text)

In [55]:
#use regex to clean and filter out the useful info from tweets related to family 
#also filter out the word related to us election
regex_election = re.compile(r'(?i)Trump|Biden|Election|democratic|republican|party|President|campaign|elector|candidate') 
regex_vaccine = re.compile(r'(?i)vaccine|Vaccine|vaccinated')

def vaccine_related_text(text):
    if regex_vaccine.search(text):
        return True
    return False

def election_related_text(text):
    if regex_election.search(text):
        return True
    return False

total_df['election_related'] = total_df['processed_text'].apply(election_related_text)
total_df['vaccine_related'] = total_df['processed_text'].apply(vaccine_related_text)

In [59]:
vaccine_related_df = total_df[(total_df["vaccine_related"]==True)&(total_df["election_related"]==False)]

In [60]:
vaccine_related_df.head(3)

,created_at,text,id_x,geo_id,full_name,country_code,name,day_index,week_index,States,processed_text,election_related,vaccine_related
82,2020-03-21T02:20:27.000Z,They should name the vaccine to the corona vir...,1241187889116897280,fbd6d2f5a4e4a15e,"California, USA",US,California,3,0,CA,They should name the vaccine to the corona vir...,False,True
125,2020-03-22T18:51:18.000Z,I honestly believe the corona virus is in the ...,1241799632914546688,fbd6d2f5a4e4a15e,"California, USA",US,California,5,0,CA,I honestly believe the corona virus is in the ...,False,True
440,2020-04-05T23:47:34.000Z,"Any vaccination is a NO for me, except for the...",1246947619064008705,fbd6d2f5a4e4a15e,"California, USA",US,California,18,2,CA,"Any vaccination is a NO for me, except for the...",False,True


In [61]:
provaccine = ['had both of my shots','Covid vaccine next month will solve that problem','Get the damned vaccines','Finally got my mother a vaccine appointment','take the first vaccine','ain’t taking NO vaccine','got first dose of Pfizer vaccine','It has been an incredible week in research','took the vaccine','need my vaccine','Good info on how mRNA vaccines will work','just got the COVIDvaccine','got my second shot','Vaccines here go through rigorous testing and are entirely safe','hope to be in the first phase of the vaccinated','entered a trial','Got into the Pfizer trial','take the vaccine under normal circumstances','pro-vaccine','I have full confidence in our scientists','Great to see the news','Hopefully more good news is too come','wait for a vaccine','keep me in my house until there’s an actual vaccine','join a COVID-19 vaccine trial','need a vaccine for COVID 19','have a vaccine','an effective vaccine administered to a large percentage of the population','vaccine works','Why NOT rush Covid-19 vaccines','Vaccine’d ROLLING','incredible first look at the COVID-19 vaccine','got the Moderna vaccine','Banned an anti-vaxxer','got my vaccine','got the first COVID19 vaccine','got my 1st round of the covid vaccine','Fully vaccinated','privilege in getting the vaccine early','Waiting for my drive-thru COVID vaccine','getting the covid vaccine','Make the vaccine mandatory','Vaccine needs to be mandatory','dont worry about the vaccine','got the covid vaccine','get your vaccine','Received 1st dose of the COVID19 vaccine','Vaccine day, pretty excited!','vaccine 1 DONE!','received my first shot of Covid vaccine','Volunteering at a COVID vaccine hub','get his first dose of the vaccine','get your vaccine','waiting for my Covid vaccine','getting my second vaccine shot','2nd vaccine done COVID 19 ','got my covid vaccine','getting my Vaccine','got my first shot of the Pfizer','vaccine received''When can I get the COVID-19 vaccine?','2nd dose of the Pfizer vaccine is now inside of me','Finally got the vaccine','wait for her vaccine','Got my second dose of the Covid vaccine','approved for my covid vaccine ','for their first Covid vaccine','First dose of the COVID-19 vaccine']
antivaccine=['Any vaccination is a NO for me','No vaccines for me','refusing to take the vaccine','antivaccine','don’t need the vaccine','vaccine is going spoil','don’t trust those vaccines','Vaccines are not needed','skeptical on the vaccine','Why would i take the vaccine if i’m not sick','ain’t taking no vaccines','No vaccine necessary','not taking the vaccine','Time to defund vaccines','anti any Gates-supported Covid19 vaccine',' will not get the vaccine','won’t get a vaccine','dont give one','he doesnt like vaccines','vaccine would certainly only be available for the purpose of profiteering','interested in the Pfizer vaccine','found shit for COVID','There’s no vaccine for Covid-19','No Vaccine Needed','Give me COVID19 instead','I won’t take a vaccine','Donno what is in it','vaccines are Unnecessary','not gonna take their stupid vaccines','recent exciting news for two mRNA vaccines','H-word for the vaccine','vaccines UNNECESSARY','don’t trust vaccines','Vaccines are destroying','Vaccine conspiracy','vaccine won’t be necessary','anti-vaxxers','Vaccines cause most mental illnesses','No vaccine']

In [62]:
regexpprovaccine = re.compile(r"(?=("+'|'.join(provaccine)+r"))")
def provaccinematchcount(text):
    return len(re.findall(regexpprovaccine, text))
vaccine_related_df['regex_poscount'] = vaccine_related_df['processed_text'].apply(provaccinematchcount)

regexpantivaccine = re.compile(r"(?=("+'|'.join(antivaccine)+r"))")
def antivaccinematchcount(text):
    return len(re.findall(regexpantivaccine, text))
vaccine_related_df['regex_negcount'] = vaccine_related_df['processed_text'].apply(antivaccinematchcount)


/tmp/ipykernel_30801/210289294.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vaccine_related_df['regex_poscount'] = vaccine_related_df['processed_text'].apply(provaccinematchcount)
/tmp/ipykernel_30801/210289294.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vaccine_related_df['regex_negcount'] = vaccine_related_df['processed_text'].apply(antivaccinematchcount)


In [63]:
def posneg_regex(row):
    if row['regex_poscount'] > row['regex_negcount']:
        return "positive"
    elif row['regex_poscount'] < row['regex_negcount']:
        return "negative"
    return "neutral"
vaccine_related_df['posneg_regex'] = vaccine_related_df.apply(posneg_regex, axis=1)

/tmp/ipykernel_30801/3744193438.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vaccine_related_df['posneg_regex'] = vaccine_related_df.apply(posneg_regex, axis=1)


In [66]:
vaccine_related_df

,created_at,text,id_x,geo_id,full_name,country_code,name,day_index,week_index,States,processed_text,election_related,vaccine_related,regex_poscount,regex_negcount,posneg_regex
82,2020-03-21T02:20:27.000Z,They should name the vaccine to the corona vir...,1241187889116897280,fbd6d2f5a4e4a15e,"California, USA",US,California,3,0,CA,They should name the vaccine to the corona vir...,False,True,0,0,neutral
125,2020-03-22T18:51:18.000Z,I honestly believe the corona virus is in the ...,1241799632914546688,fbd6d2f5a4e4a15e,"California, USA",US,California,5,0,CA,I honestly believe the corona virus is in the ...,False,True,0,0,neutral
440,2020-04-05T23:47:34.000Z,"Any vaccination is a NO for me, except for the...",1246947619064008705,fbd6d2f5a4e4a15e,"California, USA",US,California,18,2,CA,"Any vaccination is a NO for me, except for the...",False,True,0,1,negative
607,2020-04-19T01:14:23.000Z,"Just a thought... If vaccines worked, they wou...",1251680511698694144,fbd6d2f5a4e4a15e,"California, USA",US,California,31,4,CA,"Just a thought... If vaccines worked, they wou...",False,True,0,0,neutral
644,2020-04-20T03:04:58.000Z,Until there’s a vaccine for #COVIDー19 it’s unf...,1252070728544862208,fbd6d2f5a4e4a15e,"California, USA",US,California,32,4,CA,Until there’s a vaccine for COVID19 it’s unfor...,False,True,0,0,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3591745,2021-03-03T21:59:40.000Z,Utah County Health Department has confirmed th...,1367233237567766528,0fc2f35bbc546000,Utah County Health Department,US,Utah County Health Department,350,49,UT,Utah County Health Department has confirmed th...,False,True,0,0,neutral
3591748,2021-03-03T22:21:12.000Z,yes — I did get my COVID vaccine in my Chromat...,1367238655060959235,07d9e3cc35c88002,Rockaway Townsquare,US,Rockaway Townsquare,350,49,NJ,yes — I did get my COVID vaccine in my Chromat...,False,True,0,0,neutral
3591751,2021-03-03T23:12:07.000Z,@ASlavitt This week we are celebrating we have...,1367251471704731652,07d9cf70ab883002,ama medical group,US,ama medical group,350,49,FL,This week we are celebrating we have the vacc...,False,True,0,0,neutral
3591753,2021-03-04T00:06:51.000Z,One down. So thankful! #CovidVaccine #Moderna ...,1367265244737802240,0fc342d62ed4f001,"California State University, Long Beach",US,"California State University, Long Beach",350,49,CA,One down. So thankful! CovidVaccine Moderna An...,False,True,0,0,neutral


In [71]:
sample_df = vaccine_related_df[vaccine_related_df['posneg_regex']!='neutral']
# sample_df = vaccine_related_df.sample(n=100)
sample_df = sample_df.sample(n=100)
for _,row in sample_df.iterrows():
    print("NEW TWEET")
    print(row['text'])
    print (row['posneg_regex'])
    print("\n")

NEW TWEET
@StevenFulop That isnt how the vaccine works.
positive


NEW TWEET
@TrumpRulzz They have a vaccine
positive


NEW TWEET
@jackandjill525 @RealAaronHarvey @ClayTravis Sick burn, dude! I listened to the whole thing. His claim was that the entire country should wait for a vaccine because his opinion is that the current admin’s distro plan isn’t inclusive enough, which is asinine. If the vaccine is ready, get it to the people who want it, ASAP.
positive


NEW TWEET
@mattyglesias ffs we have a vaccine that nobody said would happen so quickly being administered to millions already

JFC calm the f down
positive


NEW TWEET
“I don’t understand how the vaccine works so I’m not getting it” yeah why tf would you, a normal ass person, understand molecular biology and immunology if you haven’t studied it?? That argument makes no damn sense
positive


NEW TWEET
@FrFryer @Acosta It’s still 2020 and we have a vaccine so . . . ?
positive


NEW TWEET
Fun fact: its actually not recommended you g

In [70]:
# sample_df = vaccine_related_df[vaccine_related_df['posneg_regex']!='neutral']
sample_df = vaccine_related_df.sample(n=100)
# sample_df = sample_df.sample(n=100)
for _,row in sample_df.iterrows():
    print("NEW TWEET")
    print(row['text'])
    print (row['posneg_regex'])
    print("\n")

NEW TWEET
@Abel_0888 Footage of first person getting vaccine at Disneyland. Sign me up!!
@keklotz13 @Disneyland @DisneylandToday @WaltDisneyCo @Disney @WaltDisneyWorld @CaptainAmerica @Marvel @MarveIFacts @MarvelStudios #Disneyland  #DisneyPlus #DisneyWorld #disneyparks  #COVID19 #CovidVaccine #trend https://t.co/uuljS2novF
neutral


NEW TWEET
@whoneedsit @RepMattGaetz Rachel, please don’t take the vaccine.
neutral


NEW TWEET
A lot of people have questioned things before ultimately being forced to do something, I know a lot of people that question vaccines but are forced to get ‘em because they have to make money &amp; support their families. Why can’t they question/be wary of shit?
neutral


NEW TWEET
@Jeanne5082 We are so far behind in vaccine distribution relative to population along with our Unemployment Department not paying honest folks their own money in over a month etc. 

We are on pace for our Seniors to be vaccinated by JUNE at this pace. JUNE!
neutral


NEW TWEET
If u feel

In [ ]:
provaccine = ['had both of my shots','Covid vaccine next month will solve that problem','Get the damned vaccines','Finally got my mother a vaccine appointment','take the first vaccine','ain’t taking NO vaccine','got first dose of Pfizer vaccine','It has been an incredible week in research','took the vaccine','need my vaccine','Good info on how mRNA vaccines will work','just got the COVIDvaccine','got my second shot','Vaccines here go through rigorous testing and are entirely safe','hope to be in the first phase of the vaccinated','entered a trial','Got into the Pfizer trial','take the vaccine under normal circumstances','pro-vaccine','I have full confidence in our scientists','Great to see the news','Hopefully more good news is too come','wait for a vaccine','keep me in my house until there’s an actual vaccine','join a COVID-19 vaccine trial','need a vaccine for COVID 19','have a vaccine','an effective vaccine administered to a large percentage of the population','vaccine works','Why NOT rush Covid-19 vaccines','Vaccine’d ROLLING','incredible first look at the COVID-19 vaccine','got the Moderna vaccine','Banned an anti-vaxxer','got my vaccine','got the first COVID19 vaccine','got my 1st round of the covid vaccine','Fully vaccinated','privilege in getting the vaccine early','Waiting for my drive-thru COVID vaccine','getting the covid vaccine','Make the vaccine mandatory','Vaccine needs to be mandatory','dont worry about the vaccine','got the covid vaccine','get your vaccine','Received 1st dose of the COVID19 vaccine','Vaccine day, pretty excited!','vaccine 1 DONE!','received my first shot of Covid vaccine','Volunteering at a COVID vaccine hub','get his first dose of the vaccine','get your vaccine','waiting for my Covid vaccine','getting my second vaccine shot','2nd vaccine done COVID 19 ','got my covid vaccine','getting my Vaccine','got my first shot of the Pfizer','vaccine received''When can I get the COVID-19 vaccine?','2nd dose of the Pfizer vaccine is now inside of me','Finally got the vaccine','wait for her vaccine','Got my second dose of the Covid vaccine','approved for my covid vaccine ','for their first Covid vaccine','First dose of the COVID-19 vaccine']
antivaccine=['Any vaccination is a NO for me','No vaccines for me','refusing to take the vaccine','antivaccine','don’t need the vaccine','vaccine is going spoil','don’t trust those vaccines','Vaccines are not needed','skeptical on the vaccine','Why would i take the vaccine if i’m not sick','ain’t taking no vaccines','No vaccine necessary','not taking the vaccine','Time to defund vaccines','anti any Gates-supported Covid19 vaccine',' will not get the vaccine','won’t get a vaccine','dont give one','he doesnt like vaccines','vaccine would certainly only be available for the purpose of profiteering','interested in the Pfizer vaccine','found shit for COVID','There’s no vaccine for Covid-19','No Vaccine Needed','Give me COVID19 instead','I won’t take a vaccine','Donno what is in it','vaccines are Unnecessary','not gonna take their stupid vaccines','recent exciting news for two mRNA vaccines','H-word for the vaccine','vaccines UNNECESSARY','don’t trust vaccines','Vaccines are destroying','Vaccine conspiracy','vaccine won’t be necessary','anti-vaxxers','Vaccines cause most mental illnesses','No vaccine']
